<a href="https://colab.research.google.com/github/ankush8523/Land-Cover-Classification-in-Remote-Sensing/blob/main/landCover(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spectral

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56441 sha256=df7099ebff478982f0fcb209fb6452155e468b03d4e280d725781ff314d004d2
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [ ]:
import keras
from keras.layers import Dropout, Input, GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
# from keras.utils import np_utils
from keras import utils
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
from operator import truediv
from plotly.offline import init_notebook_mode
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import spectral
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from scipy.io import loadmat
import pandas as pd
from sklearn.preprocessing import minmax_scale
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dropout, Input, AveragePooling2D, Activation, MaxPooling2D, BatchNormalization, Concatenate
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from numpy.random import seed
from time import time
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall, Accuracy
from tensorflow.keras import optimizers, regularizers
import cv2
seed(11)
init_notebook_mode(connected=True)
%matplotlib inline

In [ ]:
import os
os.environ['KERAS_BACKEND']='tensorflow'

In [ ]:
def loadData(name):

    data_path = os.path.join(os.getcwd(),'/content/drive/MyDrive/dataset/')

    if name == 'Houston':

        MS = sio.loadmat(os.paath.join(data_path, 'HS-MS Houston2013/data_MS_HR.mat'))['data_MS_HR']
        HS= sio.loadmat(os.path.join(data_path, 'HS-MS Houston2013/data_HS_LR.mat'))['data_HS_LR']
        Train = sio.loadmat(os.path.join(data_path, 'HS-MS Houston2013/Train.mat'))['Train']
        Test = sio.loadmat(os.path.join(data_path, 'HS-MS Houston2013/Validation.mat'))['Validation']

    if name == 'Berlin':

        SAR = sio.loadmat(os.path.join(data_path, 'BerlinDataset/data_SAR_HR.mat'))['data_SAR_HR']
        HS = sio.loadmat(os.path.join(data_path, 'BerlinDataset/data_HS_LR.mat'))['data_HS_LR']
        Train= sio.loadmat(os.path.join(data_path, 'BerlinDataset/TrainImage.mat'))['TrainImage']
        Test = sio.loadmat(os.path.join(data_path, 'BerlinDataset/TestImage.mat'))['TestImage']

    if name == 'Augsburg':

        SAR = sio.loadmat(os.path.join(data_path, 'Augsburg/data_SAR_HR.mat'))['data_SAR_HR']
        HS = sio.loadmat(os.path.join(data_path, 'Augsburg/data_HS_LR.mat'))['data_HS_LR']
        DSM = sio.loadmat(os.path.join(data_path, 'Augsburg/data_DSM.mat'))['data_DSM']
        Train= sio.loadmat(os.path.join(data_path, 'Augsburg/TrainImage.mat'))['TrainImage']
        Test = sio.loadmat(os.path.join(data_path, 'Augsburg/TestImage.mat'))['TestImage']

    return SAR, HS, DSM, Train, Test


In [ ]:
## GLOBAL VARIABLES
windowSize = 8

In [ ]:
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

In [ ]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [ ]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [ ]:
def createImageCubes(X, y, windowSize=9, removeZeroLabels = True):
    margin = int((windowSize) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin , c - margin:c + margin ]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [ ]:
dataset = 'Augsburg'
SAR, HS , DSM, Train, Test = loadData(dataset)

K = 15 if dataset == 'Berlin' else 12
HSr, pca = applyPCA(HS, numComponents=K)
DSM= DSM.reshape((DSM.shape[0],DSM.shape[1],1))

In [ ]:
XTrain1, Train1 = createImageCubes(SAR, Train, windowSize=windowSize)
XTest1, Test1 = createImageCubes(SAR, Test, windowSize=windowSize)

XTrain2, Train2 = createImageCubes(HSr, Train, windowSize=windowSize)
XTest2, Test2 = createImageCubes(HSr, Test, windowSize=windowSize)

XTrain3, Train3 = createImageCubes(DSM, Train, windowSize=windowSize)
XTest3, Test3 = createImageCubes(DSM, Test, windowSize=windowSize)

XtrainS = np.concatenate((XTrain1, XTrain2, XTrain3) , axis = 3)
XtestS = np.concatenate((XTest1, XTest2, XTest3) , axis = 3)

YtrainS = Train1
YtestS = Test1
YtrainS = utils.to_categorical(YtrainS)

In [ ]:
pip install keras-cv-attention-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.3/796.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 9.2 MB/s eta 0:00:00


In [ ]:
n_outputs=7

from keras_cv_attention_models import attention_layers
image_size=8
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(XtrainS)

In [ ]:
def AttentionGCN():

    input_layer = Input(shape=(8,8,17), name='input')
    #inputs = data_augmentation(input_layer)
    conv1 = Conv2D(8, kernel_size=1, activation='relu', name='conv1', kernel_regularizer=regularizers.l2(0.0005))(input_layer)

    nn = attention_layers.cot_attention(conv1, kernel_size=3)

    #pool1 = MaxPooling2D(pool_size=(2,2), name='pool1')(nn)
    # nn1 = attention_layers.neighborhood_attention(conv1, kernel_size=3)
    # nn2 = attention_layers.neighborhood_attention(conv1,kernel_size=5)
    # concat = layers.concatenate([nn1, nn2], axis =-1)
    # drop = Dropout(0.2)(concat)
    # multi_Attent = Conv2D(8, kernel_size=1, activation='relu', name='multi_Attent', kernel_regularizer=regularizers.l2(0.0005))(drop)


# ########## Graph-based global reasoning module # input is 8x8x8

    graph_t = Reshape((8, 8*8))(nn)
    squeezed_graph_t = Conv1D(16, 1, activation='relu', name='squeezer', kernel_regularizer=regularizers.l2(0.0001))(graph_t)
    squeezed_graph = Reshape((8, 16))(squeezed_graph_t)
    gconv = Conv1D(64, 1, activation='relu', name='gconv', kernel_regularizer=regularizers.l2(0.0001))(squeezed_graph)
    gconv_t = Reshape((8, 64))(gconv)
    unsqueezed_graph_transpose = Conv1D(64, 1, activation='relu', name='unsqueezer', kernel_regularizer=regularizers.l2(0.0001))(gconv_t)
    unsqueezed_graph = Reshape((64, 8))(unsqueezed_graph_transpose)
    glore = Reshape((8, 8, 8))(unsqueezed_graph)

    # Proposed Block
    nn1 = attention_layers.neighborhood_attention(glore, kernel_size=3)
    nn2 = attention_layers.neighborhood_attention(glore, kernel_size=5)
    concat1 = layers.concatenate([nn1, nn2], axis =-1)
    drop1 = Dropout(0.2)(concat1)
    multi_Attent1 = Conv2D(8, kernel_size=1, activation='relu', name='multi_Attent1', kernel_regularizer=regularizers.l2(0.0005))(drop1)

    # nn1 =  attention_layers.neighborhood_attention(glore, kernel_size=3)

    block1 = layers.concatenate([conv1, multi_Attent1], axis=-1) #################  you do not need to add the feature maps of the conv1 layer
    block12 = layers.concatenate([conv1, block1,nn ], axis=-1) #################  you do not need to use the feature maps of the attention block
    #pool2 = MaxPooling2D(pool_size=(2,2), name='pool2')(block1)
    conv2 = Conv2D(32, kernel_size=1, activation='relu', name='conv2', kernel_regularizer=regularizers.l2(0.0005))(block12)

# Graph-based global reasoning module 2 # input is 8x8x32

    graph_t2 = Reshape((32,8*8))(conv2)
    squeezed_graph_t2 = Conv1D(64, 1, activation='relu', name='squeezer2', kernel_regularizer=regularizers.l2(0.0001))(graph_t2)
    squeezed_graph2 = Reshape((64, 32))(squeezed_graph_t2)
    gconv2 = Conv1D(32, 1, activation='relu', name='gconv2', kernel_regularizer=regularizers.l2(0.0001))(squeezed_graph2)
    gconv_transpose2 = Reshape((64, 32))(gconv2)
    unsqueezed_graph_t2 = Conv1D(8*8, 1, activation='relu', name='unsqueezer2', kernel_regularizer=regularizers.l2(0.0001))(gconv_transpose2)
    unsqueezed_graph2 = Reshape((8*8, 64))(unsqueezed_graph_t2)
    glore2 = Reshape((8, 8, 64))(unsqueezed_graph2)

    # nn2 =  attention_layers.neighborhood_attention(glore2, kernel_size=3)

    # Proposed Block
    nn3 = attention_layers.neighborhood_attention(glore2, kernel_size=3)
    nn4 = attention_layers.neighborhood_attention(glore2, kernel_size=5)
    concat2 = layers.concatenate([nn3, nn4], axis =-1)
    drop2 = Dropout(0.2)(concat2)
    multi_Attent2 = Conv2D(8, kernel_size=1, activation='relu', name='multi_Attent2', kernel_regularizer=regularizers.l2(0.0005))(drop2)

    block2 = layers.concatenate([conv2, glore2], axis=-1) #################  you do not need to add the feature maps of the conv2 layer
    block21 = layers.concatenate([conv2, block2, multi_Attent2], axis=-1) #################  you do not need to use the feature maps of the attention block2

    conv3 = Conv2D(32, kernel_size=1, activation='relu', name='conv3', kernel_regularizer=regularizers.l2(0.0005))(block21)
    conv4 = Conv2D(64, kernel_size=3, activation='relu', name='conv4', kernel_regularizer=regularizers.l2(0.0005))(conv3)
    conv5 = Conv2D(128, kernel_size=1, activation='relu', name='conv5', kernel_regularizer=regularizers.l2(0.0005))(conv4)

    # flat = Flatten()(conv5)
    gap = GlobalAveragePooling2D()(conv5)

    fc1 = Dense(100, activation='relu', name='fc1', kernel_regularizer=regularizers.l2(0.01))(gap)
    fc2 = Dense(20, activation='relu', name='fc2', kernel_regularizer=regularizers.l2(0.01))(fc1)
    fc3 = Dense(7, activation='softmax', name='fc3', kernel_regularizer=regularizers.l2(0.0001))(fc2)

    model = Model(inputs=input_layer, outputs=fc3, name='CNN_model_with_global_reasoning_module')

    return model

# define the model with input layer and output layer
model = AttentionGCN()

In [ ]:
model.summary()

Model: "CNN_model_with_global_reasoning_module"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 8, 8, 17)]           0         []                            
                                                                                                  
 conv1 (Conv2D)              (None, 8, 8, 8)              144       ['input[0][0]']               
                                                                                                  
 zero_padding2d_1 (ZeroPadd  (None, 10, 10, 8)            0         ['conv1[0][0]']               
 ing2D)                                                                                           
                                                                                                  
 conv2d_6 (Conv2D)           (None, 8, 8, 8)              144

In [ ]:
# def AttentionGCN():

#     input_layer = Input(shape=(8,8,17), name='input')
#     #inputs = data_augmentation(input_layer)
#     conv1 = Conv2D(8, kernel_size=1, activation='relu', name='conv1', kernel_regularizer=regularizers.l2(0.0005))(input_layer)
#     nn = attention_layers.cot_attention(conv1, kernel_size=3)
#     #pool1 = MaxPooling2D(pool_size=(2,2), name='pool1')(nn)

# ########## Graph-based global reasoning module # input is 8x8x8

#     graph_t = Reshape((8, 8*8))(conv1)
#     squeezed_graph_t = Conv1D(16, 1, activation='relu', name='squeezer', kernel_regularizer=regularizers.l2(0.0001))(graph_t)
#     squeezed_graph = Reshape((8, 16))(squeezed_graph_t)
#     gconv = Conv1D(64, 1, activation='relu', name='gconv', kernel_regularizer=regularizers.l2(0.0001))(squeezed_graph)
#     gconv_t = Reshape((8, 64))(gconv)
#     unsqueezed_graph_transpose = Conv1D(64, 1, activation='relu', name='unsqueezer', kernel_regularizer=regularizers.l2(0.0001))(gconv_t)
#     unsqueezed_graph = Reshape((64, 8))(unsqueezed_graph_transpose)
#     glore = Reshape((8, 8, 8))(unsqueezed_graph)

#     nn1 =  attention_layers.neighborhood_attention(glore, kernel_size=3)

#     block1 = layers.concatenate([conv1, nn1], axis=-1) #################  you do not need to add the feature maps of the conv1 layer
#     block12 = layers.concatenate([conv1, block1, nn], axis=-1) #################  you do not need to use the feature maps of the attention block
#     #pool2 = MaxPooling2D(pool_size=(2,2), name='pool2')(block1)
#     conv2 = Conv2D(32, kernel_size=1, activation='relu', name='conv2', kernel_regularizer=regularizers.l2(0.0005))(block12)

# # Graph-based global reasoning module 2 # input is 8x8x32

#     graph_t2 = Reshape((32,8*8))(conv2)
#     squeezed_graph_t2 = Conv1D(64, 1, activation='relu', name='squeezer2', kernel_regularizer=regularizers.l2(0.0001))(graph_t2)
#     squeezed_graph2 = Reshape((64, 32))(squeezed_graph_t2)
#     gconv2 = Conv1D(32, 1, activation='relu', name='gconv2', kernel_regularizer=regularizers.l2(0.0001))(squeezed_graph2)
#     gconv_transpose2 = Reshape((64, 32))(gconv2)
#     unsqueezed_graph_t2 = Conv1D(8*8, 1, activation='relu', name='unsqueezer2', kernel_regularizer=regularizers.l2(0.0001))(gconv_transpose2)
#     unsqueezed_graph2 = Reshape((8*8, 64))(unsqueezed_graph_t2)
#     glore2 = Reshape((8, 8, 64))(unsqueezed_graph2)

#     nn2 =  attention_layers.neighborhood_attention(glore2, kernel_size=3)

#     block2 = layers.concatenate([conv2, nn2], axis=-1) #################  you do not need to add the feature maps of the conv2 layer
#     block21 = layers.concatenate([conv2, block2, nn], axis=-1) #################  you do not need to use the feature maps of the attention block2

#     conv3 = Conv2D(32, kernel_size=1, activation='relu', name='conv3', kernel_regularizer=regularizers.l2(0.0005))(block21)
#     conv4 = Conv2D(64, kernel_size=3, activation='relu', name='conv4', kernel_regularizer=regularizers.l2(0.0005))(conv3)
#     conv5 = Conv2D(128, kernel_size=1, activation='relu', name='conv5', kernel_regularizer=regularizers.l2(0.0005))(conv4)

#     # flat = Flatten()(conv5)
#     gap = GlobalAveragePooling2D()(conv5)

#     fc1 = Dense(100, activation='relu', name='fc1', kernel_regularizer=regularizers.l2(0.01))(gap)
#     fc2 = Dense(20, activation='relu', name='fc2', kernel_regularizer=regularizers.l2(0.01))(fc1)
#     fc3 = Dense(7, activation='softmax', name='fc3', kernel_regularizer=regularizers.l2(0.0001))(fc2)

#     model = Model(inputs=input_layer, outputs=fc3, name='CNN_model_with_global_reasoning_module')

#     return model

# # define the model with input layer and output layer
# model = AttentionGCN()

In [ ]:
# compiling the model
adam = Adam(learning_rate=0.002)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
# checkpoint
filepath = "/content/drive/MyDrive/model/AttGCN_Hsr_Augsburg.h5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(x=XtrainS, y=YtrainS, batch_size=256, epochs=100, callbacks=callbacks_list)
model.save("/content/drive/MyDrive/model/AttGCN_Hsr_Augsburg.h5")

In [ ]:
import gc
gc.collect()

881

In [ ]:
Xtest = XtestS.reshape(-1, windowSize, windowSize, 17)
ytest = utils.to_categorical(YtestS)
y_pred_test = model.predict(Xtest)
y_pred_test = np.argmax(y_pred_test, axis=1)
kappa = cohen_kappa_score(YtestS, y_pred_test)
print("Kappa score : %.5f" % kappa)
ca = np.sum(y_pred_test == np.argmax(ytest, axis=1)) / ytest.shape[0]
print("Classification accuracy: %.5f" % ca)
classification = classification_report(np.argmax(ytest, axis=1), y_pred_test)
print(classification)

2423/2423 [==============================] - 21s 8ms/step
Kappa score : 0.84058
Classification accuracy: 0.88889
              precision    recall  f1-score   support

           0       0.95      0.95      0.95     13361
           1       0.90      0.96      0.93     30065
           2       0.59      0.59      0.59      3830
           3       0.97      0.90      0.94     26609
           4       0.14      0.56      0.22       523
           5       0.26      0.03      0.05      1638
           6       0.56      0.43      0.49      1507

    accuracy                           0.89     77533
   macro avg       0.63      0.63      0.60     77533
weighted avg       0.89      0.89      0.89     77533

